# Miscellaneous Stuff

*Version History*

* **0.0** - Implement section on centering matrix
* **0.1** - Add section covering model size (number of parameters)
* **0.2** - Address influence of shared activations when computing RDMs


## Imports

In [1]:
import torch
from torch import nn

from src.models.lenet import LeNet
from src.rsa.rdm import compute_rdm

## Centering Matrix

The [centering matrix](https://en.wikipedia.org/wiki/Centering_matrix) of dimension $n$ is defined as
$$
C_n = I_n - \frac{1}{n} J_n \,,
$$
where $I_n$ is the identity matrix and $J_n$ is a matrix of all $1$'s.

In [2]:
def get_centering_matrix(n: int) -> torch.Tensor:
    """Return the centering matrix of dimension n."""
    
    return torch.eye(n) - torch.ones(n, n) / n


### Sample Matrix

In [3]:
g = torch.Generator()
g.manual_seed(42)

sample_matrix = torch.randint(low=-10, high=10, size=(3, 4), generator=g, dtype=torch.float32)

display(sample_matrix)
print("\n\033[1mRow means\033[0m:")
display(torch.round(sample_matrix.mean(dim=1), decimals=2))
print("\n\033[1mColumn means\033[0m:")
display(torch.round(sample_matrix.mean(dim=0), decimals=2))

tensor([[ -8.,  -3.,   6.,   4.],
        [ -4.,   5., -10.,  -6.],
        [  0.,   3.,   8.,   4.]])


Row means:


tensor([-0.2500, -3.7500,  3.7500])


Column means:


tensor([-4.0000,  1.6700,  1.3300,  0.6700])

### Centering Columns

Multiplying a $(m \times n)$-matrix $A$ with the centering matrix $C_m$ **from the left**, results in a matrix
$$
C_m A = A - \frac{1}{m} J_m A \,,
$$
whose **columns have $0$ mean**.

In [4]:
sample_matrix_col = torch.mm(get_centering_matrix(sample_matrix.size(dim=0)), sample_matrix)

display(torch.round(sample_matrix_col, decimals=2))
print("\n\033[1mRow means\033[0m:")
display(torch.round(sample_matrix_col.mean(dim=1), decimals=2))
print("\n\033[1mColumn means\033[0m:")
display(torch.round(sample_matrix_col.mean(dim=0), decimals=2))

tensor([[ -4.0000,  -4.6700,   4.6700,   3.3300],
        [  0.0000,   3.3300, -11.3300,  -6.6700],
        [  4.0000,   1.3300,   6.6700,   3.3300]])


Row means:


tensor([-0.1700, -3.6700,  3.8300])


Column means:


tensor([0., -0., -0., -0.])

The same result can be achieved by using PyTorch's ``mean`` method.

In [5]:
torch.round(sample_matrix_col - sample_matrix_col.mean(dim=0, keepdim=True), decimals=2)

tensor([[ -4.0000,  -4.6700,   4.6700,   3.3300],
        [  0.0000,   3.3300, -11.3300,  -6.6700],
        [  4.0000,   1.3300,   6.6700,   3.3300]])

### Centering Rows

Multiplying a $(m \times n)$-matrix $A$ with the centering matrix $C_n$ **from the right**, results in a matrix
$$
A C_n = A - \frac{1}{n} A J_n \,,
$$
whose **rows have $0$ mean**.

In [6]:
sample_matrix_row = torch.mm(sample_matrix, get_centering_matrix(sample_matrix.size(dim=1)))

display(torch.round(sample_matrix_row, decimals=2))
print("\n\033[1mRow means\033[0m:")
display(torch.round(sample_matrix_row.mean(dim=1), decimals=2))
print("\n\033[1mColumn means\033[0m:")
display(torch.round(sample_matrix_row.mean(dim=0), decimals=2))

tensor([[-7.7500, -2.7500,  6.2500,  4.2500],
        [-0.2500,  8.7500, -6.2500, -2.2500],
        [-3.7500, -0.7500,  4.2500,  0.2500]])


Row means:


tensor([0., 0., 0.])


Column means:


tensor([-3.9200,  1.7500,  1.4200,  0.7500])

Again, we can achieve the same result using PyTorch's ``mean`` method.

In [7]:
torch.round(sample_matrix_row - sample_matrix_row.mean(dim=1, keepdim=True), decimals=2)

tensor([[-7.7500, -2.7500,  6.2500,  4.2500],
        [-0.2500,  8.7500, -6.2500, -2.2500],
        [-3.7500, -0.7500,  4.2500,  0.2500]])

### Centering Rows & Columns

Simultaneously multiplying a matrix $A$ with the appropriate centering matrices **from the left and right**, results in a matrix
$$
C_m A C_n \,,
$$
whose **rows and columns both have $0$ mean**.

In [8]:
sample_matrix_both = torch.mm(
    torch.mm(get_centering_matrix(sample_matrix.size(dim=0)), sample_matrix),
    get_centering_matrix(sample_matrix.size(dim=1))
)

display(torch.round(sample_matrix_both, decimals=2))
print("\n\033[1mRow means\033[0m:")
display(torch.round(sample_matrix_both.mean(dim=1), decimals=2))
print("\n\033[1mColumn means\033[0m:")
display(torch.round(sample_matrix_both.mean(dim=0), decimals=2))

tensor([[-3.8300, -4.5000,  4.8300,  3.5000],
        [ 3.6700,  7.0000, -7.6700, -3.0000],
        [ 0.1700, -2.5000,  2.8300, -0.5000]])


Row means:


tensor([-0., -0., -0.])


Column means:


tensor([0., -0., -0., -0.])

As before, we can achieve the same result using PyTorch's ``mean`` method.

In [9]:
torch.round(sample_matrix_both - sample_matrix_both.mean(dim=(0, 1), keepdim=True), decimals=2)

tensor([[-3.8300, -4.5000,  4.8300,  3.5000],
        [ 3.6700,  7.0000, -7.6700, -3.0000],
        [ 0.1700, -2.5000,  2.8300, -0.5000]])

## Model Parameters

In [10]:
def count_parameters(model: nn.Module) -> int:
    """Count the number of trainable parameters.

    Args:
        model: The model whose parameters are to be counted.

    Returns:
        The number of trainable parameters of the ``model``.
    """
    
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


### LeNet-5

In [11]:
print(f"LeNet has {count_parameters(LeNet()):,.0f} trainable parameters.")

LeNet has 61,706 trainable parameters.


## Computing RDMs: Shared Activations

**Euclidean distance** *is* invariant to shared activations of units between stimuli, while **correlation distance** is *not*!


### Euclidean Distance

In [12]:
# Reproducibility
g = torch.Generator()
g.manual_seed(42)

# Generate random activations for 3 stimuli (100 activations each) & compute RDM
activations = torch.randn(size=[3, 100], generator=g)
rdm = compute_rdm(activations, method="euclidean")

# Add shared activation to 37th unit, compute RDM again
activations[:, 36] += 2
rdm_shared_activation = compute_rdm(activations, method="euclidean")

# Display results
print("\033[1mRDM\033[0m:")
display(rdm)
print("\n\033[1mRDM with shared activation\033[0m:")
display(rdm_shared_activation)

RDM:


tensor([1.5818, 2.0009, 2.0277])


RDM with shared activation:


tensor([1.5818, 2.0009, 2.0277])

### Correlation Distance

In [13]:
# Reproducibility
g = torch.Generator()
g.manual_seed(42)

# Generate random activations for 3 stimuli (100 activations each) & compute RDM
activations = torch.randn(size=[3, 100], generator=g)
rdm = compute_rdm(activations, method="correlation")

# Add shared activation to 37th unit, compute RDM again
activations[:, 36] += 2
rdm_shared_activation = compute_rdm(activations, method="correlation")

# Display results
print("\033[1mRDM\033[0m:")
display(rdm)
print("\n\033[1mRDM with shared activation\033[0m:")
display(rdm_shared_activation)

RDM:


tensor([0.7955, 0.9667, 1.0579])


RDM with shared activation:


tensor([0.7457, 0.9253, 0.9936])